<a href="https://colab.research.google.com/github/nrj64/eccd_assignments/blob/main/assignments/Cleaning%20a%20dataset-student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective

The objective of this notebook is get hands-on experience on cleaning a "dirty" dataset.
Often, datasets are created from "free-text" fields. In free-text fields, data validation is not enforced and as a result, many conventions co-exist within the same column of data.
Dirty data can also ocurr when collection information from different sources. If these sources use different conventions to represent such data, additional efforts are required to homogenize it at a later stage.

## Setup

In [1]:
!pip install -q eccd_datasets pygradus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.9/223.9 kB 24.5 MB/s eta 0:00:00


In [2]:
STUDENT_NAME = "Nicolas Roselli"
COURSE_NAME = "eccd-oct23"
EXERCISE_NAME = "cleaning-a-dataset"

In [15]:
import pandas as pd
import numpy as np
from eccd_datasets import load_lingerie
from pygradus import create_exercise, check_solution
import re

In [4]:
datasets = load_lingerie()

In [5]:
datasets.keys()

dict_keys(['ae_com', 'amazon_com', 'btemptd_com', 'calvinklein_com', 'hankypanky_com', 'macys_com', 'shop_nordstrom_com', 'us_topshop_com', 'victoriassecret_com'])

## The different datasets on their own

In [6]:
datasets["ae_com"].head()

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
0,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,Ae US,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Rugged Green
1,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,Ae US,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Natural Nude
2,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,Ae US,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",True Black
3,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,Ae US,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",White
4,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,Ae US,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Royal Navy


In [7]:
datasets["amazon_com"].head()

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
0,Calvin Klein Women's Sheer Marquisette Demi Un...,$36.00,$32.40,https://www.amazon.com/-/dp/B01NAVD98J?th=1&psc=1,Calvin-Klein,Bras,Amazon US,"An unlined demi cup bra featuring sheer, sexy ...",4.5,47,"[ 72% Nylon, 28% Elastane , Imported , hook an...","30B , 30C , 30D , 30DD , 32A , 32B , 32C , 32D...","30B , 30C , 30D , 30DD , 32B , 32C , 32D , 32D...",Bare
1,Wacoal Embrace Lace Bikini Panty,$27.00,$27.00,https://www.amazon.com/-/dp/B0011YQFNK?th=1&psc=1,Wacoal,Panties,Amazon US,Embrace lace bikini offers great fit and match...,4.4,91,"[ 100% Nylon , Imported , Hand Wash , 1.2"" hig...","Small , Medium , Large , X-Large","Small , Medium , X-Large",Large
2,Wacoal Women's Slimline Seamless Minimizer Bra,$65.00,$65.00,https://www.amazon.com/-/dp/B000T3606Q?th=1&psc=1,Wacoal,Bras,Amazon US,Seamless underwire minimizer bra gives great s...,4.3,298,"[ Cups: 100% Polyester; Back: 71% Nylon, 29% S...","32DD , 32DDD , 34C , 34D , 34DD , 34DDD , 36C ...","32DD , 32DDD , 34C , 34D , 34DD , 34DDD , 36C ...",Black
3,Hanky Panky Womens Signature Lace Retro V-Kini,$36.00,$36.00,https://www.amazon.com/-/dp/B003Y6AX0Y?th=1&psc=1,Hanky-Panky,Panties,Amazon US,All-day comfort describes this figure-flatteri...,4.4,46,"[ Made in USA , All-day comfort describes this...","Small , Medium , Large , X-Large","Small , Medium , X-Large",Large
4,Wacoal Women's Red Carpet Strapless Bra,$65.00,$65.00,https://www.amazon.com/-/dp/B01CEBGQA0?th=1&psc=1,Wacoal,Bras,Amazon US,"Red Carpet full figure strapless fits great, s...",4.4,747,"[ 91% Nylon, 9% Spandex , Imported , hook and ...","30D , 30DD , 30DDD , 30G , 32C , 32D , 32DD , ...","30D , 30DD , 30DDD , 30G , 32C , 32D , 32DD , ...",Pecan


### Joining all the datasets into one

In [8]:
df = pd.concat(datasets.values())

In [9]:
df.shape

(613143, 14)

# Calculating statistics on the dataset

## Unifying Victoria's Secret

In [10]:
def unify_victoria_secret(df):
    """
    Queremos que todas las marcas relacionadas con Victoria's Secret
    tengan 'victoria's secret' como su marca en lugar de lo que tienen actualmente.
    """
    df = df.copy()
    new_string = "victoria's secret"

    # Reemplazar marcas relacionadas con Victoria's Secret por 'victoria's secret'
    # Asegúrate de que el nombre de la columna de la marca sea el correcto en tu DataFrame

    if 'brand_name' in df.columns:  # Reemplaza 'brand_name' con el nombre de la columna correcto
        related_brands = ["Victoria's Secret", "Victoria's Secret Pink", "Victorias-Secret"]  # Lista de marcas relacionadas
        df['brand_name'] = df['brand_name'].apply(lambda x: new_string if x in related_brands else x)
        return df
    else:
        print("La columna de marca no se encuentra en el DataFrame. Por favor, verifica el nombre de la columna.")




In [11]:
df_unified = unify_victoria_secret(df)

In [12]:
answer_victoria_secret =df_unified[df_unified["brand_name"] == "victoria's secret"].shape[0]
print(answer_victoria_secret)

453453


## Cleaning up the price

In this sectino we are going to transform the `price` column into a float column in USD dolars.

For this, be careful of the different formats in the data.

For simplicity, you might assume that all the prices are in USD dolars, regarding of the symbol of the currency used.

In [16]:
def clean_price(df):
    df = df.copy()

    # Eliminar caracteres no deseados y convertir a tipo float
    df['price'] = df['price'].replace({'\$': '', ',': '', ' USD': '', '–': '-', '\xa0': '', '\n': '', '\t': '', 'Now': ''}, regex=True)

    # Para manejar rangos de precios y múltiples precios, extraer el precio más bajo si hay múltiples precios
    def get_lowest_price(x):
        # Encontrar todos los números (con o sin punto decimal) en el texto
        numbers = re.findall(r'\d+(?:\.\d+)?', x)

        # Convertir los números a float y devolver el mínimo si hay al menos un número
        if numbers:
            return min(map(float, numbers))
        else:
            return np.nan  # Devolver NaN si no se encuentran números

    df['price'] = df['price'].apply(get_lowest_price)

    return df

In [17]:
df_clean = clean_price(df_unified)

In [18]:
answer_unified_price_sum = df_clean["price"].sum()
print(answer_unified_price_sum)

20191976.02


In [19]:
def low_high_product_mean(df):
    df = df.copy()

    # Asegurarse de que la columna de precios sea numérica
    df['price'] = pd.to_numeric(df['price'], errors='coerce')

    # Calcular el precio promedio por categoría
    category_mean = df.groupby('product_category')['price'].mean()

    # Encontrar la categoría con el precio promedio más bajo y más alto
    lowest_mean_category = category_mean.idxmin()
    lowest_mean = category_mean.min()

    highest_mean_category = category_mean.idxmax()
    highest_mean = category_mean.max()

    return lowest_mean, highest_mean

In [20]:
lowest_mean, highest_mean = low_high_product_mean(df_clean)
assert np.allclose(lowest_mean, 3.6203030303030)
print("highest_mean", highest_mean)

highest_mean 98.0


In [21]:

proposed_solution = {
'attempt': {
    'course_name': COURSE_NAME,
    'exercise_name': EXERCISE_NAME,
    'username': STUDENT_NAME,
},
'task_attempts': [
         {
            "name": "victoria secret",
            "answer": str(answer_victoria_secret),
         },
         {
            "name": "price unification",
            "answer": str(answer_unified_price_sum),
         },
         {
            "name": "highest mean",
            "answer": str(highest_mean),
         },
]

}
check_solution(proposed_solution)


|                    Task Name                     |       Status       |
|--------------------------------------------------|--------------------|
|--------------------------------------------------|--------------------|
|                 victoria secret                  |      Correct       |
|--------------------------------------------------|--------------------|
|                price unification                 |      Correct       |
|--------------------------------------------------|--------------------|
|                   highest mean                   |      Correct       |
|--------------------------------------------------|--------------------|
